# Getting Started

For the sake of runtime, there are only 15 invoices in the data/raw folder. To add more invoices, you can find and move more invoices from the data/temp folder 

In [57]:
#Install Dependncies -- Run the below code in terminal
%pip install --upgrade pip
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [58]:
# Configure OpenAI API Key -- Create a .env file in the root directory of your project with the following content
"OPENAI_API_KEY=your_openai_api_key_here"

'OPENAI_API_KEY=your_openai_api_key_here'

# Config

In [59]:
# Imports
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

True

In [60]:
# OpenAI API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [61]:
# Data Paths
RAW_DATA_FOLDER = "data/raw"
REPORT_OUTPUT_PATH = "output/Sales_Intelligence_Report.pdf"
PARSED_OUTPUT_PATH = "output/parsed_invoices.csv"
AGGREGATED_OUTPUT_PATH = "output/aggregations"

# Agent 1: Parser

In [62]:
# Imports

import base64
import json
import re
import time
from pdf2image import convert_from_path
from io import BytesIO
import pandas as pd


In [63]:
os.makedirs("output", exist_ok=True)

In [64]:
# Converting PDF to Image

def pdf_to_base64_image(pdf_path):
    images = convert_from_path(pdf_path, first_page=1, last_page=1)
    if not images:
        return None
    buffered = BytesIO()
    images[0].save(buffered, format="PNG")
    img_bytes = buffered.getvalue()
    return base64.b64encode(img_bytes).decode("utf-8")

In [65]:
# Clean JSON string

def clean_json_string(raw_str):
    return re.sub(r"^```json\s*|\s*```$", "", raw_str.strip(), flags=re.MULTILINE)

In [66]:
# Extracting Details from PDF

def extract_text_from_pdf(pdf_path):
    try:
        b64_image = pdf_to_base64_image(pdf_path)
        if not b64_image:
            print(f"❌ Failed to convert PDF to image: {pdf_path}")
            return None

        time.sleep(1.5)

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": (
                                "Extract the following structured fields from this invoice image: "
                                "Invoice ID, Date, Customer Name, Country to Ship To, Category, Rate, Quantity, Amount. "
                                "If no field is missing, return the result as a JSON object with each field as a key"
                                "If any field is missing, just return one word: Invalid"
                            )
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{b64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            max_tokens=1200
        )

        result = response.choices[0].message.content
        if "Invalid" in result or "invalid" in result:
            print(f"Data missing")
            return None
        try:
            cleaned = clean_json_string(result)
            json_result = json.loads(cleaned)
        except json.JSONDecodeError:
            print(f"Invalid format")
            return None

        return json_result

    except Exception as e:
        print(f"TPM Limit Reached")
        return None

In [67]:
# Parsing through all invoices

parsed_data = []
total = 0
for filename in os.listdir(RAW_DATA_FOLDER):
    if filename.lower().endswith(".pdf"):
        total += 1
        pdf_path = os.path.join(RAW_DATA_FOLDER, filename)
        print(f"📄 Processing: {filename}")
        parsed = extract_text_from_pdf(pdf_path)
        if parsed != None:
            parsed_data.append(parsed)

if parsed_data:
    df = pd.DataFrame(parsed_data)
    df.to_csv(PARSED_OUTPUT_PATH, index=False)
    print(f"Parsed {len(parsed_data)}/{total} invoices into {PARSED_OUTPUT_PATH}")
else:
    print("No valid invoices were parsed.")

📄 Processing: invoice_Aaron Hawkins_47905.pdf
📄 Processing: invoice_Aaron Hawkins_40100.pdf
📄 Processing: invoice_Aaron Hawkins_40101.pdf
📄 Processing: invoice_Aaron Hawkins_36652.pdf
📄 Processing: invoice_Aaron Hawkins_36651.pdf
📄 Processing: invoice_Aaron Hawkins_4820.pdf
📄 Processing: invoice_Aaron Bergman_39519.pdf
📄 Processing: invoice_Aaron Hawkins_6817.pdf
📄 Processing: invoice_Aaron Bergman_36260.pdf
Data missing
📄 Processing: invoice_Aaron Hawkins_38461.pdf
Data missing
📄 Processing: invoice_Aaron Bergman_36259.pdf
📄 Processing: invoice_Aaron Hawkins_37425.pdf
📄 Processing: invoice_Aaron Bergman_36258.pdf
📄 Processing: invoice_Aaron Hawkins_38460.pdf
📄 Processing: invoice_Aaron Hawkins_49674.pdf
Parsed 13/15 invoices into output/parsed_invoices.csv


# Agent 2: Sales Aggregater

In [68]:
# Imports

import matplotlib.pyplot as plt

In [69]:
os.makedirs(AGGREGATED_OUTPUT_PATH, exist_ok=True)

In [70]:
# Cleaning Amount Column

def clean_amount(value):
    if isinstance(value, str):
        return round(float(value.replace("$", "").replace(",", "").strip()))
    return round(float(value))

In [71]:
# Cleaning Columns

def preprocess_data(df):
    df["Amount"] = df["Amount"].apply(clean_amount)
    df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    df["Month"] = df["Date"].dt.to_period("M")
    return df

In [72]:
# Aggregating Data

def aggregate_and_save(df, group_by, filename):
    agg = df.groupby(group_by)["Amount"].agg(["sum", "count"]).reset_index()
    agg.columns = [group_by, "Total Revenue", "Invoice Count"]

    if group_by == "Month":
        agg = agg.sort_values(by=[group_by, "Total Revenue"], ascending=[True, False])
    else:
        agg = agg.sort_values("Total Revenue", ascending=False)

    agg.to_csv(os.path.join(AGGREGATED_OUTPUT_PATH, filename), index=False)
    print(f"Saved {filename}")

In [73]:
# Plotting Monthly Trend

def plot_monthly_trend(df):
    monthly = df.groupby("Month")["Amount"].sum().sort_index()
    monthly.plot(kind="line", marker="o", title="Monthly Revenue Trend")
    plt.xlabel("Month")
    plt.ylabel("Revenue")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(AGGREGATED_OUTPUT_PATH, "monthly_trend.png"))
    plt.close()

In [74]:
# Running the Aggregator

if not os.path.exists(PARSED_OUTPUT_PATH):
    print(f"❌ File not found: {PARSED_OUTPUT_PATH}")

df = pd.read_csv(PARSED_OUTPUT_PATH)
df = preprocess_data(df)

aggregate_and_save(df, "Category", "by_category.csv")
aggregate_and_save(df, "Country to Ship To", "by_country.csv")
aggregate_and_save(df, "Customer Name", "by_customer.csv")
aggregate_and_save(df, "Month", "by_month.csv")

plot_monthly_trend(df)

print("All aggregations and visualizations completed.")

Saved by_category.csv
Saved by_country.csv
Saved by_customer.csv
Saved by_month.csv
All aggregations and visualizations completed.


# Agent 3: Recommender Agent

In [75]:
# Read in the data
category_df = pd.read_csv(os.path.join(AGGREGATED_OUTPUT_PATH, "by_category.csv"))
country_df = pd.read_csv(os.path.join(AGGREGATED_OUTPUT_PATH, "by_country.csv"))
monthly_df = pd.read_csv(os.path.join(AGGREGATED_OUTPUT_PATH, "by_month.csv"))

In [76]:
# Combine into a prompt for OpenAI
prompt = (
    "You are a sales analyst. Analyze this sales data and give 5 actionable, smart recommendations "
    "to optimize revenue next quarter. Use evidence from data. Keep the output concise and executive-style.\n\n"
    f"Top Categories:\n{category_df.to_string(index=False)}\n\n"
    f"Top Countries:\n{country_df.to_string(index=False)}\n\n"
    f"Monthly Trend:\n{monthly_df.to_string(index=False)}\n"
)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
    max_tokens=600
)

In [77]:
# Saving Insight Texts

insights_text = response.choices[0].message.content.strip()
print(insights_text)

Based on the provided sales data, here are five actionable recommendations to optimize revenue for the next quarter:

1. **Focus on High-Value Categories:**
   - The "Phones, Technology" category generated the highest revenue (total of $33,508). Prioritize marketing and sales efforts on this category, particularly targeting the TEC-PH-5841 product, which alone contributed $10,604.

2. **Expand in High-Revenue Countries:**
   - The Democratic Republic of the Congo and Guatemala are top revenue-generating countries with single transactions. Explore potential for repeat business by establishing stronger relationships and offering incentives for bulk purchases or repeat orders.

3. **Increase Sales Frequency in the United States:**
   - Despite having the highest invoice count (9), the U.S. revenue ($4,163) is relatively low. Implement targeted promotions or bundled offers to increase average order value and drive higher revenue per transaction.

4. **Leverage Low-Performing Categories:**


# Agent 4: Report Generator

In [78]:
# Imports

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage, Table, TableStyle
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

In [79]:
# Styling

doc = SimpleDocTemplate(REPORT_OUTPUT_PATH, pagesize=A4)
styles = getSampleStyleSheet()
elements = []

elements.append(Paragraph("Sales Performance Report", styles['Title']))
elements.append(Spacer(1, 12))

In [80]:
# Timeframe Calculator
months_df = pd.read_csv(os.path.join(AGGREGATED_OUTPUT_PATH, "by_month.csv"))
start = months_df['Month'].min()
end = months_df['Month'].max()
elements.append(Paragraph(f"Reporting Period: <b>{start}</b> to <b>{end}</b>", styles['Normal']))
elements.append(Spacer(1, 12))

In [81]:
# Top 5 Product Categories
top_categories = pd.read_csv(os.path.join(AGGREGATED_OUTPUT_PATH, "by_category.csv")).head(5)
elements.append(Paragraph("Top 5 Product Categories by Revenue:", styles['Heading2']))
cat_table_data = [list(top_categories.columns)] + top_categories.values.tolist()
cat_table = Table(cat_table_data, hAlign='LEFT')
cat_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
]))
elements.append(cat_table)
elements.append(Spacer(1, 12))

In [82]:
# Top 5 Countries
top_countries = pd.read_csv(os.path.join(AGGREGATED_OUTPUT_PATH, "by_country.csv")).head(5)
elements.append(Paragraph("Top 5 Countries by Revenue:", styles['Heading2']))
country_table_data = [list(top_countries.columns)] + top_countries.values.tolist()
country_table = Table(country_table_data, hAlign='LEFT')
country_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
]))
elements.append(country_table)
elements.append(Spacer(1, 12))

In [83]:
# Monthly Trend
elements.append(Paragraph("Monthly Revenue Trend:", styles['Heading2']))
trend_chart = os.path.join(AGGREGATED_OUTPUT_PATH, "monthly_trend.png")
if os.path.exists(trend_chart):
    elements.append(RLImage(trend_chart, width=400, height=250))
    elements.append(Spacer(1, 12))

In [84]:
# Recommendations
elements.append(Paragraph("Recommendations", styles['Heading2']))
clean_insights = insights_text.replace("**", "")
for para in clean_insights.split("\n"):
    if para.strip():
        elements.append(Paragraph(para.strip(), styles['Normal']))
        elements.append(Spacer(1, 6))

doc.build(elements)
print(f"Sales report PDF created: {REPORT_OUTPUT_PATH}")


Sales report PDF created: output/Sales_Intelligence_Report.pdf
